In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("worksheet4.ipynb")

# Worksheet 4: Aggregations and ACID transaction in MongoDB

## Establish Pymongo connection

Recall that in worksheet1, we have created a cluster on MongoDB Atlas and connect to it via Pymongo and a credentials JSON file. Let's reestablish that connection for our exercises.

- Make sure you use the `adsc_3610` conda environment.
- You might need to copy & paste the `credentials_mongodb.json` file that you used in worksheet1 to the working directory of worksheet3.

In [ ]:
from pymongo import MongoClient # import mongo client to connect
import json # import json to load credentials
import urllib.parse

# load credentials from json file
with open('credentials_mongodb.json') as f:
    login = json.load(f)

# assign credentials to variables
username = login['username']
password = urllib.parse.quote(login['password'])
host = login['host']
url = "mongodb+srv://{}:{}@{}/?retryWrites=true&w=majority".format(username, password, host)

In [ ]:
# connect to the database
client = MongoClient(url)

In [ ]:
# drop database books and students if they exist
client.drop_database('bookstore')
client.drop_database('students')

To test if your connection has been succesful, let's try to print out all the databases

In [ ]:
# list all databases
client.list_database_names()

- Load `sample_mflix` database into an object called db
- Load the `movies` collection into an object called `collection`

In [ ]:
db = ...
collection = ...

In [ ]:
...


In [ ]:
grader.check("ex0")

#### Exercise 1: Calculate the Average Rating of Movies by Genre

rubric: {accuracy = 2}

**Task**: Write an aggregation pipeline to calculate the average imdb rating of movies grouped by genre.

**Hint**: First, use `$unwind` stage to unwind the `genres` array before grouping by genre. Use the `$group` stage to group by genre and the `$avg` operator to calculate the average rating.



Expected output should look something like this:

```json
{'_id': 'War', 'averageRating': 7.128591954022989}
{'_id': 'News', 'averageRating': 7.252272727272728}
{'_id': 'Romance', 'averageRating': 6.6564272782136396}
...
{'_id': 'Sci-Fi', 'averageRating': 6.123609653725079}
{'_id': 'Film-Noir', 'averageRating': 7.397402597402598}
```

In [ ]:
# Exercise 1: Calculate the Average Rating of Movies by Genre

pipeline1 = [
    ...
]

# Execute the pipeline
result1 = list(collection.aggregate(pipeline1))
for doc in result1:
    print(doc)

In [ ]:
grader.check("ex1")



#### Exercise 2: Find the Top 5 Movies with the Highest IMDb Rating

rubric: {accuracy = 3}

**Task**: Write an aggregation pipeline to find the top 5 movies with the highest Tomatoes Viewer rating. In the result, only keep the movies `title` and `tomatoes.viewer.rating` field only, remove `_id` from the result.

**Hint**: Use the `$sort` stage to sort by Tomatoes Viewer rating in descending order and the `$limit` stage to limit the results to the top 5. Use `$project` to select fields that you want to include/exclude in the output.



Expected output:

```json
...
{'title': 'Murder by Natural Causes', 'tomatoes': {'viewer': {'rating': 5.0}}}
{'title': 'I Am Maria', 'tomatoes': {'viewer': {'rating': 5.0}}}
```

In [ ]:
# print out tomatoes viewer rating of some movies
for doc in collection.find({}, {"_id": 0,
                                "title": 1, 
                                "tomatoes.viewer.rating": 1}).limit(5):
    print(doc)

In [ ]:
# Exercise 2: Find the Top 5 Movies with the Highest Tomatoes Viewer Rating

pipeline2 = [
    ...
]

# Execute the pipeline
result2 = list(collection.aggregate(pipeline2))
for doc in result2:
    print(doc)

In [ ]:
grader.check("ex2")



#### Exercise 3: Count the Number of Movies by Year

rubric: {accuracy = 2}

**Task**: Write an aggregation pipeline to count the number of movies released each year. Sort by count in descending order

**Hint**: Use the `$group` stage to group by year and the `$count` operator to count the number of movies. Use the `sort` stage to sort by count



Expected output:
```json
...
{'_id': 2008, 'count': 886}
{'_id': 2010, 'count': 866}
...
```

In [ ]:
# Exercise 3: Count the Number of Movies by Year

pipeline3 = [
    ...
]

# Execute the pipeline
result3 = list(collection.aggregate(pipeline3))
for doc in result3:
    print(doc)

In [ ]:
grader.check("ex3")



#### Exercise 4: Find the Top 3 Directors with the Most Movies

rubric: {accuracy = 4}

**Task**: Write an aggregation pipeline to find the top 3 directors who have directed the most movies.

**Hint**: Use the `$group` stage to group by director, the `$count` operator to count the number of movies, the `$sort` stage to sort by the count in descending order, and the `$limit` stage to limit the results to the top 3.

You might need to `$unwind` the directors array.



Expected output:
```json
{'_id': ..., 'movieCount': ...}
{'_id': ..., 'movieCount': ...}
{'_id': ..., 'movieCount': ...}
```

In [ ]:
# Exercise 4: Find the Top 3 Directors with the Most Movies

pipeline4 = [
    ...
]

# Execute the pipeline
result4 = list(collection.aggregate(pipeline4))
for doc in result4:
    print(doc)

In [ ]:
grader.check("ex4")

### Exercise 5: Calculate the Average IMDb Rating and Number of Movies for Each Genre (Rounded to 2 Decimals)

rubric: {accuracy = 3}

**Task**: Write an aggregation pipeline to calculate the average IMDb rating (rounded to 2 decimals) and the number of movies for each genre.

**Hint**: Use the `$unwind` stage to deconstruct the genres array, the `$group` stage to group by genre, the `$avg` operator to calculate the average rating, and the `$project` stage to round the average rating to 2 decimal places.



Expected output:

```json
...
{'_id': 'Crime', 'movieCount': 2457, 'averageRating': 6.69}
{'_id': 'Sci-Fi', 'movieCount': 958, 'averageRating': 6.12}
...
```

In [ ]:
# Exercise 5: Calculate the Average IMDb Rating and Number of Movies for Each Genre (Rounded to 2 Decimals)

pipeline5 = [
    ...
]

# Execute the pipeline
result5 = list(collection.aggregate(pipeline5))
for doc in result5:
    print(doc)

In [ ]:
grader.check("ex5")

### Exercise 6: Implementing ACID Transactions for a Shopping Cart

rubric: {accuracy = 4}

In this exercise, you will implement an ACID (Atomicity, Consistency, Isolation, Durability) transaction using MongoDB. The goal is to ensure that a series of operations either all succeed or all fail, maintaining the integrity of the database.

#### Instructions

1. **Setup MongoDB Client**: Ensure you have the MongoDB client installed and connected to your database.
2. **Create Collections**: Create two collections in your MongoDB database: `inventory` and `carts`.
3. **Insert Sample Data**:
    - Insert sample items into the `inventory` collection with fields `item_id` and `quantity`.
    - Insert a sample user into the `carts` collection with fields `user_id` and `items`.
4. **Implement the Transaction**:
    - Write a function `add_to_cart` that performs the following operations within a transaction:
        - Check if the inventory has enough quantity for the specified item.
        - Deduct the quantity from the inventory.
        - Add the item to the user's shopping cart.
    - Ensure that if any operation fails, the transaction is aborted and no changes are made to the database.
5. **Run the Transaction**: Execute the transaction and handle any errors that may occur.




In [ ]:
from pymongo.errors import ConnectionFailure, OperationFailure
from pymongo.write_concern import WriteConcern
from pymongo.read_concern import ReadConcern
from pymongo.read_preferences import ReadPreference

client.drop_database('shop')  # drop the shop database if it exists

# Select the database and collections
db = client['shop']
inventory = db['inventory']
carts = db['carts']

# Insert sample data into the inventory collection
inventory.insert_many([
    {'item_id': 'item456', 'quantity': 10},
    {'item_id': 'item789', 'quantity': 5},
    {'item_id': 'item123', 'quantity': 20}
])

# Insert sample data into the carts collection
carts.insert_one({
    'user_id': 'user123',
    'items': []
})


#### 6.1 Write a function to check if the inventory has enough quantity for a particular item
- raise ValueError if the order_quantity exceed the inventory quantity
- Print out a message `Item {item_id} has enough quantity` if the order_quantity <= the inventory quantity

In [ ]:
def check_quantity(item_id, order_quantity):
    ...


In [ ]:
# print out the quantity of item123 from the inventory collection
print(inventory.find_one({'item_id': 'item123'}))


In [ ]:
check_quantity('item123', 10) # this should print out a message that item123 has enough quantity

In [ ]:
check_quantity('item123', 100) # this should raise a ValueError

#### 6.2 Write a function to update the quantity from the inventory

- Use the `update_one` function to update the quantity of inventory
- You can use the `$inc` to increment the quantity of inventory based on the order quantity

In [ ]:
def update_quantity(order_quantity, item_id):
    ...


In [ ]:
# print out the quantity of item_id 'item123' before updating
print(inventory.find_one({'item_id': 'item123'}))

In [ ]:

# Update the quantity for item_id 'item123'
order_quantity = 10
item_id = 'item123'
update_quantity(order_quantity, item_id)

In [ ]:
# print out the quantity of item_id 'item123' after updating
print(inventory.find_one({'item_id': 'item123'}))

#### 6.3 Write a function to update user's shopping cart

- Use the `update_one` function to update the shopping cart
- Use the `$push` operator to an element to an array. If the array does not exist, it will be created.

In [ ]:
# print out shopping cart before adding items
print(carts.find_one({'user_id': 'user123'}))

In [ ]:
# write a function to add items to the shopping cart
def add_to_cart(user_id, item_id, quantity):
    ...

In [ ]:
# test the function by adding items to the shopping cart
add_to_cart('user123', 'item123', 5)

In [ ]:
# print out shopping cart after adding items
print(carts.find_one({'user_id': 'user123'}))

#### 6.4 Combine everything into a transaction function

Create a transaction function that:
- First check if the quantity is enough in the inventory, if not, it should raise a ValueError
- Update the quantity from the inventory by substracting the order_quantity
- Update user's shopping cart with the item_id and the order_quantity

Note: You need to add a line `session=session` in the update_one function for inventory and shopping carts

In [ ]:

# Define the transaction function
def add_to_cart(session, user_id, item_id, quantity):
    # Check if the inventory has enough quantity
    # Deduct the quantity from the inventory 
    # Add the item to the user's shopping cart
          
    ...


In [ ]:

# Start a session
with client.start_session() as session:
    try:
        # Start a transaction
        session.start_transaction()

        # Perform the add to cart operation
        add_to_cart(session, 'user123', 'item456', 2)

        # Commit the transaction
        session.commit_transaction()
        print("Transaction committed.")
    except (ConnectionFailure, OperationFailure, ValueError) as e:
        # Abort the transaction on error
        session.abort_transaction()
        print(f"Transaction aborted due to error: {e}")

Let's test the constraint of the transaction. This should throws an error because order_quantity of 200 is exceeding the inventory quantity

In [ ]:

# Start a session
with client.start_session() as session:
    try:
        # Start a transaction
        session.start_transaction()

        # Perform the add to cart operation
        add_to_cart(session, 'user123', 'item456', 200)

        # Commit the transaction
        session.commit_transaction()
        print("Transaction committed.")
    except (ConnectionFailure, OperationFailure, ValueError) as e:
        # Abort the transaction on error
        session.abort_transaction()
        print(f"Transaction aborted due to error: {e}")

**There are no auto-tests for Q6, it will be graded manually**

## Submission instructions

{rubric: mechanics = 5}

- Make sure the notebook can run from top to bottom without any error. Restart the kernel and run all cells.
- Commit and push your notebook to the github repo
- Double check your notebook is rendered properly on Github and you can see all the outputs clearly